In [2]:
from __future__ import division
from collections import defaultdict
from collections import Counter

import os
import random
import json

In [3]:

source_dir = "C:\\Users\\Molly\\Google Drive\\senior classes\\nlp\\term_project\\book-nlp-master\\book-nlp-master\\data\\originalTexts"
# for filename in os.listdir(source_dir):
#     print filename
#     bookfile = open(source_dir + "\\" + filename, 'r')
#     book = bookfile.read()
#     bookDict = defaultdict(float)
#     for word in book.split():
#         bookDict[word] += 1
#     counter = Counter(bookDict)
#     top_n = counter.most_common(100)
#     print top_n
bookfile = open(source_dir + '\\full_hp.txt', 'r')
book = bookfile.read()
bookDict = defaultdict(float)
for word in book.split():
    bookDict[word] += 1
counter = Counter(bookDict)
top_n = counter.most_common(200)
print top_n
print

[('the', 47552.0), ('to', 26268.0), ('and', 25179.0), ('of', 21444.0), ('a', 19852.0), ('.', 17618.0), ('was', 15155.0), ('he', 14435.0), ('his', 13612.0), ('said', 12682.0), ('in', 11952.0), ('Harry', 11326.0), ('had', 9954.0), ('that', 8840.0), ('at', 8322.0), ('you', 8044.0), ('it', 7310.0), ('I', 7282.0), ('as', 7026.0), ('with', 6272.0), ('on', 6097.0), ('for', 5193.0), ('He', 4990.0), ('not', 4666.0), ('\xc2\x91', 4527.0), ('her', 4418.0), ('they', 4259.0), ('be', 4258.0), ('were', 4155.0), ('but', 4010.0), ('from', 3997.0), ('have', 3970.0), ('out', 3847.0), ('him', 3806.0), ('\xe2\x80\x94', 3561.0), ('she', 3462.0), ('up', 3432.0), ('into', 3427.0), ('Ron', 3366.0), ('all', 3336.0), ('The', 3250.0), ('been', 3136.0), ('Hermione', 3119.0), ('what', 2798.0), ('could', 2668.0), ('back', 2595.0), ('who', 2581.0), ('their', 2497.0), ('is', 2491.0), ('them', 2482.0), ('Harry,', 2400.0), ('\xe2\x80\x9cI', 2326.0), ('looked', 2321.0), ('about', 2282.0), ('so', 2241.0), ('this', 2197.0)

In [4]:
wk_dir = os.path.dirname(os.path.realpath('__file__'))

#json_filename = os.path.join(wk_dir, "..", "booknlp_output\\potter.all.book.txt")
json_filename = "C:\\Users\\Molly\\Google Drive\\senior classes\\nlp\\term_project\\char-net\\booknlp_output\potter.all.book";

with open(json_filename) as json_file:
	char_json = json.load(json_file)
print "successfully loaded json"

successfully loaded json


In [5]:
# TODO: add stopword removal (this is temporary, pull words from a file or something?)
import nltk

stopwords = nltk.corpus.stopwords.words('english')
curDir = os.path.dirname(os.path.realpath('__file__'))
stop = os.path.join(curDir,'stopwords')     
stopfile = open(stop, 'r')
stop = stopfile.read()
for word in stop.split():
    stopwords.append(word)
print stopwords
stopwords = set(stopwords)

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [6]:
# dictionary of lists of character attributes, indexed by name (i.e. character's "bag of words")
characters = {}

for character in char_json["characters"]:
    # TODO: throw out characters that are mentioned less than 10 times (play with this threshold?)
    if character["NNPcount"] > 20:
        char_name = character["names"][0]['n']
        characters[char_name] = []
        for word_dict in character["patient"]:
            curWord = word_dict['w']
            if curWord not in stopwords:
                characters[char_name].append(curWord + "_PATIENT")
        for word_dict in character["agent"]:
            curWord = word_dict['w']
            if curWord not in stopwords:
                characters[char_name].append(curWord + "_AGENT")
        for word_dict in character["mod"]:
            curWord = word_dict['w']
            if curWord not in stopwords:
                characters[char_name].append(curWord + "_MOD")
                
        #TODO: add dialogue features?
#         for word_dict in character['speaking']:
#             print word_dict['w']
                
        # are things characters possess indicative of their character? Often not, so maybe exclude
        for word_dict in character["poss"]:
            characters[char_name].append(word_dict['w'] + "_POSS")

texts = [None for i in range(len(characters))]
names = defaultdict(float)
i = 0
for key, value in characters.iteritems():
	print str(key) + ": " + str(value)
	texts[i] = value
	names[key] = i
	i += 1
	print

Madam Rosmerta: [u'forced_PATIENT', u'gasped_PATIENT', u'breathed_PATIENT', u'staring_AGENT', u'filling_AGENT', u'eject_wizard_AGENT', u'tottering_AGENT', u'send_bottle_AGENT', u'have_other_AGENT', u'saw_AGENT', u'tip_you_AGENT', u'help_[Malfoy/Lucius Malfoy/Mr. Malfoy]_AGENT', u'admitted_AGENT', u'sounded_AGENT', u'have_cold_AGENT', u'let_sigh_AGENT', u'able_MOD', u'mead_POSS', u'pub_POSS', u'bathroom_POSS', u'messages_POSS', u'talking_POSS', u'voice_POSS', u'voice_POSS', u'voice_POSS', u'nose_POSS']

Fang: [u'take_PATIENT', u'swept_PATIENT', u'bless_PATIENT', u'take_PATIENT', u'fended_PATIENT', u'leave_PATIENT', u'carrying_PATIENT', u'pushed_PATIENT', u'bowled_PATIENT', u'stroking_PATIENT', u'found_PATIENT', u'want_PATIENT', u'transfixed_PATIENT', u'started_AGENT', u'bounded_AGENT', u'let_AGENT', u'yelped_AGENT', u'kept_AGENT', u'whimpering_AGENT', u'fighting_AGENT', u'thudded_AGENT', u'howling_AGENT', u'howling_AGENT', u'fling_himself_AGENT', u'trembling_AGENT', u'ter_AGENT', u'shov

In [7]:
from gensim import corpora, models, similarities 
 
# create Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

# create bag of words model to perform analysis
corpus = [dictionary.doc2bow(text) for text in texts]

numtopics = 4

lda = models.LdaModel(corpus, num_topics=numtopics, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=100, 
                            passes=100)


In [8]:
print(lda.show_topics())

[u'0.003*left_PATIENT + 0.002*son_POSS + 0.002*wants_AGENT + 0.002*left_AGENT + 0.002*squeaked_PATIENT + 0.001*tell_I_AGENT + 0.001*disappeared_AGENT + 0.001*eyes_POSS + 0.001*vanished_AGENT + 0.001*saw_PATIENT', u'0.016*eyes_POSS + 0.015*wand_POSS + 0.011*face_POSS + 0.008*voice_POSS + 0.007*office_POSS + 0.007*hand_POSS + 0.006*head_POSS + 0.005*turned_AGENT + 0.004*desk_POSS + 0.004*feet_POSS', u'0.014*face_POSS + 0.012*eyes_POSS + 0.010*wand_POSS + 0.009*hand_POSS + 0.008*voice_POSS + 0.007*head_POSS + 0.005*turned_AGENT + 0.004*mouth_POSS + 0.004*hands_POSS + 0.004*feet_POSS', u'0.013*eyes_POSS + 0.013*head_POSS + 0.012*wand_POSS + 0.012*face_POSS + 0.009*thought_AGENT + 0.009*hand_POSS + 0.009*felt_AGENT + 0.007*feet_POSS + 0.007*voice_POSS + 0.006*saw_AGENT']


In [9]:
import numpy as np

# topics_matrix = lda.show_topics(num_topics=20, formatted=False, num_words=50)
# topics_matrix = np.array(topics_matrix)

# topic_words = topics_matrix[:,:,1]
# for i in topic_words:
#     print count
#     print([str(word) for word in i])
#     print()

for i in range(0, numtopics):
    print i
    print lda.show_topic(i, 30)
    print

0
[(0.0029403500097370475, u'left_PATIENT'), (0.0019435336984923047, u'son_POSS'), (0.0018881246445161921, u'wants_AGENT'), (0.0015852873544006769, u'left_AGENT'), (0.0015258910272241562, u'squeaked_PATIENT'), (0.0014683331668124348, u'tell_I_AGENT'), (0.0013960045757437048, u'disappeared_AGENT'), (0.0013749895009768699, u'eyes_POSS'), (0.0013695041879573789, u'vanished_AGENT'), (0.0012982673263884842, u'saw_PATIENT'), (0.0012165728236669046, u'going_AGENT'), (0.0010839992282680659, u'go_AGENT'), (0.0010832616080922422, u'roared_PATIENT'), (0.00107617285627266, u'leave_PATIENT'), (0.0010656066594834392, u'followed_PATIENT'), (0.0010367495599231417, u'face_POSS'), (0.0010039393001871591, u'girlfriend_POSS'), (0.00098060308782890672, u'died_AGENT'), (0.00097135614152287701, u'room_POSS'), (0.00093847962340520288, u'appeared_AGENT'), (0.00092196241178775181, u'kept_AGENT'), (0.0009035146130352105, u'squeaked_AGENT'), (0.00084827356831436093, u'grave_POSS'), (0.00084703083791062297, u'thin

In [10]:
topic_matrix = [[0 for i in range(numtopics)] for namekey in names]
print len(topic_matrix)
i = 0
for namekey in names:
    print str(namekey) + " topic distribution: "
    for (key, value) in lda[corpus[names[namekey]]]:
        topic_matrix[i][key] = value
    i+=1
    print lda[corpus[names[namekey]]]

159
Madam Rosmerta topic distribution: 
[(2, 0.9713476580482876)]
Fang topic distribution: 
[(3, 0.99390288070362598)]
Hogsmeade topic distribution: 
[(0, 0.021353931041750353), (1, 0.021855652886663728), (2, 0.93533749854887116), (3, 0.021452917522714741)]
Lupin topic distribution: 
[(1, 0.99854409136547218)]
Dudley topic distribution: 
[(2, 0.95615059952275661), (3, 0.043259977075444864)]
Cho topic distribution: 
[(3, 0.9951136431557297)]
Roger Davies topic distribution: 
[(0, 0.83887868996619885), (3, 0.14693604116437137)]
Dolohov topic distribution: 
[(0, 0.23267707347894212), (3, 0.75201740840452946)]
Madam Hooch topic distribution: 
[(2, 0.9847212866152939)]
Ravenclaw topic distribution: 
[(0, 0.95505788614874221), (1, 0.014970678277193953), (2, 0.014570790488657691), (3, 0.015400645085406164)]
Dad topic distribution: 
[(3, 0.97373303332138039)]
Lavender Brown topic distribution: 
[(0, 0.011431490639000987), (1, 0.37153248573306175), (2, 0.60498998993402742), (3, 0.01204603369390

In [11]:
import numpy as np

data = np.array(topic_matrix)

from sklearn.cluster import KMeans
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(data)
clusters = kmeans.labels_.tolist()
print clusters
# "The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters."
goodness = metrics.silhouette_score(data, kmeans.labels_, metric='euclidean')
print "silhouette score: " + str(goodness)
# within-cluster sum of squares
print "inertia: " + str(kmeans.inertia_)

[0, 3, 0, 2, 0, 3, 1, 3, 0, 1, 3, 0, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 2, 3, 1, 2, 2, 0, 1, 2, 0, 3, 0, 0, 2, 0, 1, 0, 0, 0, 2, 1, 2, 0, 3, 3, 0, 1, 2, 0, 0, 3, 2, 0, 3, 3, 3, 1, 1, 2, 0, 2, 3, 2, 1, 2, 0, 0, 0, 0, 3, 0, 0, 2, 0, 3, 2, 0, 3, 0, 3, 0, 1, 3, 0, 0, 2, 1, 0, 0, 3, 3, 0, 3, 2, 3, 1, 2, 3, 2, 3, 3, 2, 3, 1, 0, 1, 3, 3, 3, 0, 0, 2, 2, 3, 3, 0, 0, 3, 1, 0, 3, 2, 3, 1, 0, 3, 1, 2, 2, 3, 1, 2, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 1, 3, 2, 2, 0, 1, 3, 1, 2, 3, 1]


NameError: name 'metrics' is not defined

In [ ]:
dic={}
i=0
for nameKey in names:
    dic[str(nameKey)]=clusters[i]
    i += 1
d=sorted(dic.items(), key=lambda x:x[1])
for name in d:
    print name

In [ ]:
import pandas as pd
tempNameList = [nameKey for nameKey in names]
charClusts = {'characters': tempNameList, 'clusters': clusters}
frame = pd.DataFrame(charClusts, index = [clusters] , columns = ['character', 'clusters'])

In [ ]:
frame['clusters'].value_counts() 

In [ ]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS
from sklearn.metrics.pairwise import euclidean_distances

# get euclidean distance between each character's topic vector
dist = euclidean_distances(data, data)

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
cluster_colors = {0: 'b', 1: 'g', 2: 'r', 3: 'c'}
#set up cluster names using a dict
cluster_names = {0: '0', 
                 1: '1',
                 2: '2', 
                 3: '3', 
                }

In [ ]:
#The following is taken word for word from this tutorial: http://brandonrose.org/clustering

-

In [ ]:
#uncomment the below to save the plot if need be
plt.savefig('clusters_small_noaxes.png', dpi=200)
plt.close()

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram

linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 50)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=charClusts['characters']);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters